In [55]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'TLMN'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [56]:
from Base.TLMN.env import __ACTIONS__

@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions = np.where(validActions == 1)[0]

    actionTypes = np.zeros_like(validActions)
    for i in range(len(validActions)):
        actionTypes[i] = __ACTIONS__[validActions[i]][0]

    actionType1 = validActions[np.where(actionTypes==1)[0]]
    actionType2 = validActions[np.where(actionTypes==2)[0]]
    actionType3 = validActions[np.where(actionTypes==3)[0]]
    actionType4 = validActions[np.where(actionTypes==4)[0]]
    actionType5 = validActions[np.where(actionTypes==5)[0]]
    actionType6 = validActions[np.where(actionTypes==6)[0]]
    actionType7 = validActions[np.where(actionTypes==7)[0]]
    actionType8 = validActions[np.where(actionTypes==8)[0]]
    actionType9 = validActions[np.where(actionTypes==9)[0]]
    actionType10 = validActions[np.where(actionTypes==10)[0]]
    actionType11 = validActions[np.where(actionTypes==11)[0]]
    actionType12 = validActions[np.where(actionTypes==12)[0]]
    actionType13 = validActions[np.where(actionTypes==13)[0]]
    actionType14 = validActions[np.where(actionTypes==14)[0]]
    actionType15 = validActions[np.where(actionTypes==15)[0]]

    actionTypes = [
        actionType1,
        actionType2,
        actionType3,
        actionType4,
        actionType5,
        actionType6,
        actionType7,
        actionType8,
        actionType9,
        actionType10,
        actionType11,
        actionType12,
        actionType13,
        actionType14,
        actionType15 
    ]

    if np.sum(state[119:113]) == 0:
        if len(actionType3) > 1:
            return actionType3[0], per

    if np.min(state[107:110]) >= 9:
        if 0 in validActions:
            return 0, per

    for i in (15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1):
        if (len(actionTypes[i-1]) == 1) and (len(validActions)==1):
            return actionTypes[i-1][0], per

    for i in (13, 12, 11, 10,  9,  7,  6,  5,  8):
        if len(actionTypes[i-1])>0:
            return actionTypes[i-1][0], per
        
    for i in (3, 2, 1, 14, 15, 4):
        if len(actionTypes[i-1])>0:
            return actionTypes[i-1][0], per
    
    action = validActions[np.random.randint(len(validActions))]
    return action, per

In [58]:
win, per = numba_main_2(player, 10000, np.array([]), 1)
win

4693